# Project: Youtube Analyzer
## Authors: Karmehr Arora, Alfred Koh

### Project Overview
Data: Youtube API’s
GUI: Tkinter

Roles:

    Karmehr: Accessing Data from the API & organizing it

    Alfred: Analyzing the data & representing it with tkinter

Goal: 
Finding the top 10 trending basketball videos (or videos of a requested topic) in the last 24 hours
Look for the most (total likes + total comments + total shares) under tag = basketball in the last 24 hours

Limitations: 
Estimated limit of searches of 50 per day due to Youtube API usage restrictions


In [19]:
# import utilized libraries pertaining to data collection
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [20]:
# get access key to API
from getpass import getpass
API_KEY = getpass('Enter your API Key:')

In [21]:
# define the api service and version
api_service_name = 'youtube'
api_version = 'v3'

## Requesting User to choose topic
default to basketball as topic if user has error/opts out of entering topic

In [22]:
# request user for topic (default = 'basketball')
topic = 'basketball'
topic = input()

## Search youtube for videos with that tag

In [23]:
# access the youtube api client
youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey = API_KEY)

# make a request to the api to find videos under that topic
# note: cost of this operation is 100 units of the alloted 10,000 units per 24 hours
request = youtube.search().list(
        part="id,snippet",
        maxResults=50,
        q=topic
    )

In [24]:
# execute the query in the request
response = request.execute()

In [25]:
print("Search ID: ", response["items"][0]['id'])
print("\nSnippet Data:",response["items"][0]['snippet'])
videos = response["items"]
count = 1
for video in videos:
    if('videoId' in video['id']):
        print(count, ": https://www.youtube.com/watch?v=" + video['id']['videoId'])
        count = count+1
    

Search ID:  {'kind': 'youtube#video', 'videoId': 'MjKZkmwd-TI'}

Snippet Data: {'publishedAt': '2023-04-03T17:10:34Z', 'channelId': 'UC86mbDclXQeljk975_KZpXg', 'title': 'Revenge Moments in Football 😈', 'description': '', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/MjKZkmwd-TI/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/MjKZkmwd-TI/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/MjKZkmwd-TI/hqdefault.jpg', 'width': 480, 'height': 360}}, 'channelTitle': 'Bsm Edits 20', 'liveBroadcastContent': 'none', 'publishTime': '2023-04-03T17:10:34Z'}
1 : https://www.youtube.com/watch?v=MjKZkmwd-TI
2 : https://www.youtube.com/watch?v=nOOubgYvo3g
3 : https://www.youtube.com/watch?v=umcSzeIamJI
4 : https://www.youtube.com/watch?v=747YKtj6d9E
5 : https://www.youtube.com/watch?v=P9DVNB3HPLw
6 : https://www.youtube.com/watch?v=YAsIDEUVqE4
7 : https://www.youtube.com/watch?v=wrIljVGTbkY
8 : https://www.youtube.com/

## Calculate and Find top 10 most trending videos under that tag

In [27]:
collection_of_video_data = {}
for video in videos:
    if('videoId' in video['id']):
        request = youtube.videos().list(part = 'snippet, statistics', id = video['id']['videoId'])
        response = request.execute()
        collection_of_video_data[video['id']['videoId']] = response

In [35]:
# video_id_to_trend_total = []
#for vid_data in collection_of_video_data:
# print(collection_of_video_data.values())
    # trendTotal = (vid_data['items'][0]['statistics']['viewCount'] + vid_data['items'][0]['statistics']['likeCount'] + 
                #  vid_data['items'][0]['statistics']['commentCount'])
    # print(trendTotal)

#     for video in videos:
#         video_id = video['id']['videoId']
#         video_id_to_trend_total[video_id].append(trendTotal)
# print(video_id_to_trend_total)


dict_values([{'kind': 'youtube#videoListResponse', 'etag': 'IH584ocf6RVq7zsC4WnG6X90Us8', 'items': [{'kind': 'youtube#video', 'etag': 'rMSDldxRJ6J75fCYME9WCuCoJYc', 'id': 'MjKZkmwd-TI', 'snippet': {'publishedAt': '2023-04-03T17:10:34Z', 'channelId': 'UC86mbDclXQeljk975_KZpXg', 'title': 'Revenge Moments in Football 😈', 'description': '', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/MjKZkmwd-TI/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/MjKZkmwd-TI/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/MjKZkmwd-TI/hqdefault.jpg', 'width': 480, 'height': 360}, 'standard': {'url': 'https://i.ytimg.com/vi/MjKZkmwd-TI/sddefault.jpg', 'width': 640, 'height': 480}, 'maxres': {'url': 'https://i.ytimg.com/vi/MjKZkmwd-TI/maxresdefault.jpg', 'width': 1280, 'height': 720}}, 'channelTitle': 'Bsm Edits 20', 'categoryId': '17', 'liveBroadcastContent': 'none', 'defaultLanguage': 'en', 'localized': {'title': 'Revenge M